In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import simpy
import gym
from gym import spaces
import matplotlib.cm as cm
import pickle
import os
import pickle
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, Dropout, MaxPooling2D, Activation, Flatten
from collections import deque
import tensorflow as tf
from tensorflow import keras

# from gym.spaces import Discrete, Box

In [2]:


# Config
# log-distance loss model parameters

PTX = 14.0
GAMMA = 2.08
D0 = 40.0
VAR = 0  # variance ignored for now
LPLD0 = 127.41

# Gate way localisation
BSX = 0.0
BSY = 0.0
HM = 1.0  # m
HB = 15.0  # m

RAY = 700.0

# Created Nodes
NODES = []

packetlen = 20


BW = 125
PL = 20
CR = 1

p = 300
AVGSENDTIME = p
# Inter-SF Thresshold Mateix

IS7 = np.array([6, -8, -9, -9, -9, -9])
IS8 = np.array([-11, 6, -11, -12, -13, -13])
IS9 = np.array([-15, -13, 6, -13, -14, -15])
IS10 = np.array([-19, -18, -17, 6, -17, -18])
IS11 = np.array([-22, -22, -21, -20, 6, -20])
IS12 = np.array([-25, -25, -25, -24, -23, 6])
SIR = np.array([IS7, IS8, IS9, IS10, IS11, IS12])


config_dict = {
    0: {"sf": 7, "sens": -126.5, "snr": -7.5},
    1: {"sf": 8, "sens": -127.25, "snr": -10},
    2: {"sf": 9, "sens": -131.25, "snr": -12.5},
    3: {"sf": 10, "sens": -132.75, "snr": -15},
    4: {"sf": 11, "sens": -133.25, "snr": -17.5},
    5: {"sf": 12, "sens": -134.5, "snr": -20},
}

ch = [868100000, 868300000, 868500000]
tpx = [2, 5, 8, 11, 14]
MODEL = 6
SEED = 913

rho_max = 0.5
tp_max = 14
tp_min = 2

N = 1000


TX = [
    22,
    22,
    22,
    23,  # RFO/PA0: -2..1
    24,
    24,
    24,
    25,
    25,
    25,
    25,
    26,
    31,
    32,
    34,
    35,
    44,  # PA_BOOST/PA1: 2..14
    82,
    85,
    90,  # PA_BOOST/PA1: 15..17
    105,
    115,
    125,
]  # PA_BOOST/PA1+PA2: 18..20


In [3]:


def SF_alloc_ch_utilization(NODES):
    sorted_nodes = []
    sorted_nodes = sorted(NODES, key=lambda x: x.dist, reverse=False)
    c_class = np.zeros(6, dtype=int)
    ch_len = len(ch)
    ch_u = np.zeros(ch_len, dtype=int)
    # print(c_class)
    for n in sorted_nodes:
        sf = n.sf
        # print(sf)
        config = config_dict[sf - 7]
        # print(config)
        for i in range(sf, 12):
            if n.snr < config["snr"] or n.rssi < config["sens"]:
                if n.sf < 12:
                    n.sf = n.sf + 1
                    n.packet.sf = n.sf
                    n.update_rectime(n.sf)
            config = config_dict[n.sf - 7]
        c_class[n.sf - 7] = c_class[n.sf - 7] + 1
        ch_index = ch.index(n.freq)
        ch_u[ch_index] = ch_u[ch_index] + 1
    return sorted_nodes, c_class, ch_u


def compute_sf_ch_utilization(nodes):
    sf_dist = np.zeros(6, dtype=int)
    tp_dist = np.zeros(5, dtype=int)
    ch_len = len(ch)
    ch_dist = np.zeros(ch_len, dtype=int)
    # print(ch_dist)
    # print(ch_len)
    # print_nodes(nodes)
    for n in nodes:
        sf = n.sf
        freq = n.freq
        tx = n.tx
        sf_dist[sf - 7] = sf_dist[sf - 7] + 1
        tp_index = tpx.index(tx)
        # print('index',tp_index)
        tp_dist[tp_index] = tp_dist[tp_index] + 1

        # print('Channel : ',freq)
        ch_index = ch.index(freq)
        # print('index',ch_index)
        ch_dist[ch_index] = ch_dist[ch_index] + 1
    return sf_dist, tp_dist, ch_dist


def energy_consumption(nodes):
    energy = (
        sum(
            node.packet.rectime * TX[int(node.tx) + 2] * 3 * node.sent for node in nodes
        )
        / 1e6
    )
    return energy


def print_nodes(nodes):
    for n in nodes:
        print("node_id: {} SF: {} TP: {} CH: {} RSSI: {} SNR: {}".format(n.id, n.sf, n.tx, n.freq,n.rssi,n.snr))


def reward_plot(rewards):
    plt.plot(rewards)
    plt.show()


In [4]:
class Packet:
    def __init__(self, nodeid, sf, bw, freq, rssi, rectime):
        self.nodeid = nodeid
        self.pl = packetlen
        self.arriveTime = 0
        # Reception Parameters
        self.sf = sf
        self.bw = bw
        self.freq = freq
        self.rssi = rssi
        self.rectime = rectime
        self.collided = 0
        self.processed = 0


In [5]:



random.seed(SEED)


class Device:
    def __init__(self, id):
        self.id = id
        self.x = 0
        self.y = 0
        self.dist = 0
        found = False
        while not found:
            a = random.random()
            b = random.random()
            if b < a:
                a, b = b, a
            posx = b * RAY * math.cos(2 * math.pi * a / b) + BSX
            posy = b * RAY * math.sin(2 * math.pi * a / b) + BSY
            if len(NODES) > 0:
                for index, n in enumerate(NODES):
                    dist = np.sqrt(((abs(n.x - posx)) ** 2) + ((abs(n.y - posy)) ** 2))
                    if dist >= 20:
                        found = 1
                        self.x = posx
                        self.y = posy
                        break
            else:
                self.x = posx
                self.y = posy
                found = True

        dist_2d = np.sqrt(
            (self.x - BSX) * (self.x - BSX) + (self.y - BSY) * (self.y - BSY)
        )
        # self.dist = np.sqrt((dist_2d)**2+(HB-HM)**2)
        self.dist = dist_2d
        # Radio Parameters
        self.bw = BW
        self.sf = 7
        self.tx = PTX
        self.cr = 1
        self.freq = random.choice(ch)
        # self.freq = 868100000
        self.rssi = self.tx - self.estimatePathLoss(MODEL)
        self.snr = random.randrange(-10, 20)
        self.rectime = self.airtime(self.sf, self.cr, packetlen, BW)
        self.packet = Packet(
            self.id, self.sf, self.bw, self.freq, self.rssi, self.rectime
        )
        self.sent = 0
        self.received = 0

    def airtime(self, sf, cr, pl, bw):
        H = 0  # implicit header disabled (H=0) or not (H=1)
        DE = 0  # low data rate optimization enabled (=1) or not (=0)
        Npream = 8  # number of preamble symbol (12.25  from Utz paper)

        if bw == 125 and sf in [11, 12]:
            # low data rate optimization mandated for BW125 with SF11 and SF12
            DE = 1
        if sf == 6:
            # can only have implicit header with SF6
            H = 1
        Tsym = (2.0 ** sf) / bw  # msec
        Tpream = (Npream + 4.25) * Tsym
        # print "sf", sf, " cr", cr, "pl", pl, "bw", bw
        payloadSymbNB = 8 + max(
            math.ceil((8.0 * pl - 4.0 * sf + 28 + 16 - 20 * H) / (4.0 * (sf - 2 * DE)))
            * (cr + 4),
            0,
        )
        Tpayload = payloadSymbNB * Tsym
        return (Tpream + Tpayload) / 1000.0  # in seconds

    def log_distance_loss(self, dist):
        Lpl = LPLD0 + 10 * GAMMA * math.log10(dist / D0)
        rssi = PTX - Lpl
        return rssi

    def update_rectime(self, sf):
        self.rectime = self.airtime(sf, self.cr, packetlen, BW)
        self.packet.rectime = self.rectime

    def estimatePathLoss(self, model):
        # Log-Distance model
        if model == 0:
            Lpl = LPLD0 + 10 * GAMMA * math.log10(self.dist / D0)

        # Okumura-Hata model
        elif model >= 1 and model <= 4:
            # small and medium-size cities
            if model == 1:
                ahm = (
                    1.1 * (math.log10(self.freq) - math.log10(1000000)) - 0.7
                ) * HM - (1.56 * (math.log10(self.freq) - math.log10(1000000)) - 0.8)

                C = 0
            # metropolitan areas
            elif model == 2:
                if self.freq <= 200000000:
                    ahm = 8.29 * ((math.log10(1.54 * HM)) ** 2) - 1.1
                elif self.freq >= 400000000:
                    ahm = 3.2 * ((math.log10(11.75 * HM)) ** 2) - 4.97
                C = 0
            # suburban enviroments
            elif model == 3:
                ahm = (
                    1.1 * (math.log10(self.freq) - math.log10(1000000)) - 0.7
                ) * HM - (1.56 * (math.log10(self.freq) - math.log10(1000000)) - 0.8)

                C = -2 * ((math.log10(self.freq) - math.log10(28000000)) ** 2) - 5.4
            # rural area
            elif model == 4:
                ahm = (
                    1.1 * (math.log10(self.freq) - math.log10(1000000)) - 0.7
                ) * HM - (1.56 * (math.log10(self.freq) - math.log10(1000000)) - 0.8)

                C = (
                    -4.78 * ((math.log10(self.freq) - math.log10(1000000)) ** 2)
                    + 18.33 * (math.log10(self.freq) - math.log10(1000000))
                    - 40.98
                )

            A = (
                69.55
                + 26.16 * (math.log10(self.freq) - math.log10(1000000))
                - 13.82 * math.log(HB)
                - ahm
            )

            B = 44.9 - 6.55 * math.log10(HB)

            Lpl = A + B * (math.log10(self.dist) - math.log10(1000)) + C

        # 3GPP model
        elif model >= 5 and model < 7:
            # Suburban Macro
            if model == 5:
                C = 0  # dB
            # Urban Macro
            elif model == 6:
                C = 3  # dB

            Lpl = (
                (44.9 - 6.55 * math.log10(HB))
                * (math.log10(self.dist) - math.log10(1000))
                + 45.5
                + (35.46 - 1.1 * HM) * (math.log10(self.freq) - math.log10(1000000))
                - 13.82 * math.log10(HM)
                + 0.7 * HM
                + C
            )

        # Polynomial 3rd degree
        elif model == 7:
            p1 = -5.491e-06
            p2 = 0.002936
            p3 = -0.5004
            p4 = -70.57

            Lpl = (
                p1 * math.pow(self.dist, 3)
                + p2 * math.pow(self.dist, 2)
                + p3 * self.dist
                + p4
            )

        # Polynomial 6th degree
        elif model == 8:
            p1 = 3.69e-12
            p2 = 5.997e-11
            p3 = -1.381e-06
            p4 = 0.0005134
            p5 = -0.07318
            p6 = 4.254
            p7 = -171

            Lpl = (
                p1 * math.pow(self.dist, 6)
                + p2 * math.pow(self.dist, 5)
                + p3 * math.pow(self.dist, 4)
                + p4 * math.pow(self.dist, 3)
                + p5 * math.pow(self.dist, 2)
                + p6 * self.dist
                + p7
            )

        return Lpl


In [6]:

### Collision Detection ################
"""

This Part Allows Collision Checking Between Two Packets :
 1- Timing Collision
 2- Frequency Collision
 3- SF Collision
 4- Capture Effect
 5- Imperfect SF Orthogonality
 
"""
########################### 1-Timining Collision #################


def timingCollision(env, p1, p2):
    Npream = 8
    Tpreamb = 2 ** p1.sf / (1.0 * p1.bw) * (Npream - 5)
    p2_end = p2.addTime + p2.rectime

    p1_cs = env.now + (Tpreamb / 1000.0)  # to sec

    """ print ("collision timing node {} ({},{},{}) node {} ({},{})".format(
        p1.nodeid, env.now - env.now, p1_cs - env.now, p1.rectime,
        p2.nodeid, p2.addTime - env.now, p2_end - env.now
    ))"""
    if p1_cs < p2_end:
        # p1 collided with p2 and lost
        # print ("not late enough")
        return True
    # print ("saved by the preamble")
    return False


######################### 2-Frequency Collision #######################
def frequencyCollision(p1, p2):
    if abs(p1.freq - p2.freq) <= 120 and (p1.bw == 500 or p2.freq == 500):
        # print ("frequency coll 500")
        return True
    elif abs(p1.freq - p2.freq) <= 60 and (p1.bw == 250 or p2.freq == 250):
        # print( "frequency coll 250")
        return True
    else:
        if abs(p1.freq - p2.freq) <= 30:
            # print ("frequency coll 125")
            return True
        # else:
    # print ("no frequency coll")
    return False


####################### 3- SF Collision ###############################
def sfCollision(p1, p2):
    if p1.sf == p2.sf:
        # print ("collision sf node {} and node {}".format(p1.nodeid, p2.nodeid))
        return True
    # print ("no sf collision")
    return False


####################### 4-5 ############################################
def powerCollision_2(p1, p2):
    # powerThreshold = 6
    # print ("SF: node {0.nodeid} {0.sf} node {1.nodeid} {1.sf}".format(p1, p2))
    # print ("pwr: node {0.nodeid} {0.rssi:3.2f} dBm node {1.nodeid} {1.rssi:3.2f} dBm; diff {2:3.2f} dBm".format(p1, p2, round(p1.rssi - p2.rssi,2)))

    if p1.sf == p2.sf:

        if abs(p1.rssi - p2.rssi) < 6:
            # print ("collision pwr both node {} and node {}".format(p1.nodeid, p2.nodeid))
            # packets are too close to each other, both collide
            # return both packets as casualties
            """print(
                "power coll  cap : freq 1 : {} and freq 2 :{} ==> |{}| < {}".format(
                    p1.freq, p2.freq, abs(p1.rssi - p2.rssi), SIR[p1.sf - 7][p2.sf - 7]
                )
            )"""
            return (p1, p2)
        elif p1.rssi - p2.rssi < 6:
            # p2 overpowered p1, return p1 as casualty
            # print ("collision pwr node {} overpowered node {}".format(p2.nodeid, p1.nodeid))
            # print ("capture - p2 wins, p1 lost")
            """print(
                "power coll  cap : freq 1 : {} and freq 2 :{}  => {} < {}".format(
                    p1.freq, p2.freq, p1.rssi - p2.rssi, SIR[p1.sf - 7][p2.sf - 7]
                )
            )"""
            return (p1,)
        # print ("capture - p1 wins, p2 lost")
        # p2 was the weaker packet, return it as a casualty
        """print(
            "power coll  cap : freq 1 : {} and freq 2 :{} => {} > {}".format(
                p1.freq, p2.freq, p1.rssi - p2.rssi, SIR[p1.sf - 7][p2.sf - 7]
            )
        )"""
        return (p2,)
    else:

        if p1.rssi - p2.rssi > SIR[p1.sf - 7][p2.sf - 7]:

            # print ("P1 is OK")
            if p2.rssi - p1.rssi > SIR[p2.sf - 7][p1.sf - 7]:
                # print ("p2 is OK")
                return ()
            else:
                # print ("p2 is lost")
                # print("power coll  Imp : 2")
                return (p2,)

        else:

            # print ("p1 is lost")
            if p2.rssi - p1.rssi > SIR[p2.sf - 7][p1.sf - 7]:

                # print ("p2 is OK")
                # print("power coll  Imp : 1")
                return (p1,)
            else:
                # print ("p2 is lost")
                # print("opwer coll  cap : 1")
                return (p1, p2)


######################## Check Collision ############################
def checkcollision(env, packet, packetsAtBS, maxBSReceives, full_collision):
    col = 0  # flag needed since there might be several collisions for packet
    processing = 0
    for i in range(0, len(packetsAtBS)):
        if packetsAtBS[i].packet.processed == 1:
            processing = processing + 1
    if processing > maxBSReceives:
        # print ("too long:", len(packetsAtBS))
        packet.processed = 0
    else:
        packet.processed = 1

    if packetsAtBS:
        # print ("CHECK node {} (sf:{} bw:{} freq:{:.6e}) others: {}".format(
        # packet.nodeid, packet.sf, packet.bw, packet.freq,
        # len(packetsAtBS)))
        # print(len(packetsAtBS))
        for other in packetsAtBS:

            if other.id != packet.nodeid:
                # print (">> node {} (sf:{} bw:{} freq:{:.6e})".format(
                # other.id, other.packet.sf, other.packet.bw, other.packet.freq))
                if full_collision == 1 or full_collision == 2:

                    if frequencyCollision(packet, other.packet) and timingCollision(
                        env, packet, other.packet
                    ):
                        # check who collides in the power domain
                        if full_collision == 1:
                            # Capture effect
                            c = powerCollision_2(packet, other.packet)
                        else:
                            # Capture + Non-orthognalitiy SFs effects
                            c = powerCollision_2(packet, other.packet)
                        # mark all the collided packets
                        # either this one, the other one, or both
                        for p in c:
                            p.collided = 1
                            if p == packet:
                                col = 1

                    else:
                        # no freq or timing collision, all fimone
                        pass
                else:
                    # simple collision
                    if frequencyCollision(packet, other.packet) and sfCollision(
                        packet, other.packet
                    ):
                        packet.collided = 1
                        other.packet.collided = (
                            1  # other also got lost, if it wasn't lost already
                        )
                        col = 1
        return col
    return 0


In [7]:

# Node generation and Plot Function


def node_Gen(nrNodes, display):
    NODES = []
    x = []
    y = []
    rssi = []
    dist = []

    for i in range(nrNodes):
        device = Device(i)
        NODES.append(device)

        x.append(NODES[i].x)
        y.append(NODES[i].y)
        rssi.append(NODES[i].rssi)
        dist.append(NODES[i].dist)
    pickle.dump(NODES, open("data/nodes.p", "wb"))
    if display:
        fig, ax = plt.subplots()
        ax.plot(x, y, "ro")
        ax.plot(0, 0, "bo")
        for i in range(6):
            v1 = np.linspace((-RAY / 6) * (i + 1), (RAY / 6) * (i + 1), 100)
            v2 = np.linspace((-RAY / 6) * (i + 1), (RAY / 6) * (i + 1), 100)
            X, Y = np.meshgrid(v1, v2)
            F = X ** 2 + Y ** 2 - ((RAY / 6) * (i + 1)) ** 2

            ax.contour(X, Y, F, [0], colors="k", linestyles="dashed", linewidths=1)
        ax.set_aspect(1)
        plt.show()

    return NODES


# SF Allocation Display

# RA-Lora Plot
def SF_alloc_plot(sorted_nodes, exp, k, display=False, save=False):
    groups = []
    for s in range(7, 13):
        group = []
        posx = []
        posy = []
        for n in sorted_nodes:
            if n.sf == s:
                posx.append(n.x)
                posy.append(n.y)

        group.append(posx)
        group.append(posy)
        groups.append(group)
        group = []
    # print(groups)
    colors = ["ro", "go", "bo", "yo", "co", "mo"]
    # print(len(groups[3][0]))
    # print(len(groups[3][1]))
    fig, ax = plt.subplots()
    for g, c in zip(groups, colors):
        ax.plot(g[0], g[1], c)

    ax.plot(0, 0, "ko")

    for i in range(6):
        v1 = np.linspace((-RAY / 6) * (i + 1), (RAY / 6) * (i + 1), 100)
        v2 = np.linspace((-RAY / 6) * (i + 1), (RAY / 6) * (i + 1), 100)
        X, Y = np.meshgrid(v1, v2)
        F = X ** 2 + Y ** 2 - ((RAY / 6) * (i + 1)) ** 2

        ax.contour(X, Y, F, [0], colors="k", linestyles="dashed", linewidths=1)
    ax.set_aspect(1)
    if save:
        if not os.path.exists("reports/graphics/{}".format(exp)):
            os.mkdir("reports/graphics/{}".format(exp))
        plt.savefig("reports/graphics/{}/sf_alloc_{}".format(exp, k))
    if display:
        plt.show()
    plt.close()

    
def finalReport(data):
    fname = "sim_results-v2-1.csv"
    if not os.path.isfile("reports/results/{}".format(fname)):
        df_new = pd.DataFrame(data, index=[0])
        df_new.to_csv("reports/results/{}".format(fname), index=False)
    else:
        df = pd.read_csv("reports/results/{}".format(fname))
        df_new = pd.DataFrame(data, index=[df.ndim - 1])
        df = df.append(df_new, ignore_index=True)
        df.to_csv("reports/results/{}".format(fname), index=False)


def airtime(sf, cr, pl, bw):
    H = 0  # implicit header disabled (H=0) or not (H=1)
    DE = 0  # low data rate optimization enabled (=1) or not (=0)
    Npream = 8  # number of preamble symbol (12.25  from Utz paper)

    if bw == 125 and sf in [11, 12]:
        # low data rate optimization mandated for BW125 with SF11 and SF12
        DE = 1
    if sf == 6:
        # can only have implicit header with SF6
        H = 1
    Tsym = (2.0 ** sf) / bw  # msec
    Tpream = (Npream + 4.25) * Tsym
    # print "sf", sf, " cr", cr, "pl", pl, "bw", bw
    payloadSymbNB = 8 + max(
        math.ceil((8.0 * pl - 4.0 * sf + 28 + 16 - 20 * H) / (4.0 * (sf - 2 * DE)))
        * (cr + 4),
        0,
    )
    Tpayload = payloadSymbNB * Tsym
    return (Tpream + Tpayload) / 1000.0  # in seconds
        
        
def mathematical_model(N, sf_dist, avgtime):
    charges = []
    DERs = []
    s = 0
    for i in range(6):
        Nsf = sf_dist[i]
        toa = airtime(i + 7, 1, packetlen, BW)
        c = (Nsf * toa) / avgtime
        charges.append(c)
        DERsf = math.exp(-2*c)
        DERs.append(DERsf)
        s = s + (DERsf * Nsf)
    DER = s / N
    return DER, DERs, charges

In [8]:



class Environment(gym.Env):
    def __init__(self, nodes, SF_dist_init, Ch_dist_init):

        """ Environment Parameter"""

        super(Environment, self).__init__()

        self.NODES = nodes
        self.N = len(self.NODES)
        self.TP_SET = [2, 5, 8, 11, 14]
        self.SF_SET = [7, 8, 9, 10, 11, 12]
        self.CH_SET = ["868.1", "868.3", "868.5"]
        self.ACTION_SPACE_GEN = self.action_space_generator()
        self.ACTION_SPACE_SIZE = len(self.ACTION_SPACE_GEN)
        self.STATE_SPACE = []
        self.SF_DIST = SF_dist_init
        self.ENERGY = 1
        self.CH_DIST = Ch_dist_init

        """ Simulation Parameters"""
        self.action_space = spaces.Box(
            np.array([0,7, 0, 0]), np.array([self.N - 1,12, 4, 2]), dtype=np.int32
        )
        # self.observation_space = spaces.Box(np.array([0,7,0,0,-10,-135]), np.array([N,12,4,2,10,-70]),dtype =np.int)
        self.observation_space = spaces.Box(
            np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
            np.array(
                [
                    self.N,
                    self.N,
                    self.N,
                    self.N,
                    self.N,
                    self.N,
                    self.N,
                    self.N,
                    self.N,
                    self.N,
                    self.N,
                    self.N,
                    self.N,
                    self.N,
                ]
            ),
            dtype=np.int32,
        )
        self.condition = 0

    def action_space_generator(self):
        action_space = []

        for n in range(self.N):

            for sf in self.SF_SET:

                for tp in range(len(self.TP_SET)):

                    for ch in range(len(self.CH_SET)):
                        action_space.append([n, sf, tp, ch])
        return action_space

    def toa(self, sf, cr, pl, bw):
        H = 0  # implicit header disabled (H=0) or not (H=1)
        DE = 0  # low data rate optimization enabled (=1) or not (=0)
        Npream = 8  # number of preamble symbol (12.25  from Utz paper)

        if bw == 125 and sf in [11, 12]:
            # low data rate optimization mandated for BW125 with SF11 and SF12
            DE = 1
        if sf == 6:
            # can only have implicit header with SF6
            H = 1
        Tsym = (2.0 ** sf) / bw  # msec
        Tpream = (Npream + 4.25) * Tsym
        # print "sf", sf, " cr", cr, "pl", pl, "bw", bw
        payloadSymbNB = 8 + max(
            math.ceil((8.0 * pl - 4.0 * sf + 28 + 16 - 20 * H) / (4.0 * (sf - 2 * DE)))
            * (cr + 4),
            0,
        )
        Tpayload = payloadSymbNB * Tsym
        return (Tpream + Tpayload) / 1000.0  # in seconds

    def rho_compute(self, sf, n):
        return (self.toa(sf, CR, PL, BW) * n) / p

    def reward_function(self, rho, u_ch, tp):
        return (rho_max) / (rho * u_ch) + ((tp_max - tp) / (tp_max - tp_min))

    def reward_function_zero(self, tp):
        return (tp_max - tp) / (tp_max - tp_min)

    def compute_der(self, sf_dist):
        der = 0
        for index in range(len(sf_dist)):
            rho = self.rho_compute(self.SF_SET[index], sf_dist[index])
            der_sf = math.exp(-2 * rho)
            der = der + (der_sf * sf_dist[index])
        return (der / self.N) * 100

    def reset(self):
        sf_dist, tp_dist, u_ch = compute_sf_ch_utilization(self.NODES)
        state = np.concatenate((sf_dist, tp_dist, u_ch))
        self.condition = 0
        return state

    def step(self, action):
        # print('Step : ',self.condition)
        # print('Action : ',action)
        # assert self.action_space.contains(action)
        reward = 0
        #node_index = random.randint(0,self.N-1)
        node_index = action[0]
        node = self.NODES[node_index]
        old_sf = node.sf
        old_tp = node.tx
        old_ch = node.freq
        new_sf = action[1]
        new_tp = self.TP_SET[action[2]]
        new_ch = action[3]

        sf_dist, tp_dist, u_ch = compute_sf_ch_utilization(self.NODES)
        der = self.compute_der(sf_dist)
        #print("DER : ", der)
        config = config_dict[new_sf - 7]
        # print("config : ", config)
        
        # print('rho',rho)
        # print("node snr : {} rssi : {}".format(node.snr, node.rssi))
        # print("node old sf : {} new sf: {}".format(old_sf, new_sf))
        if node.snr < config["snr"] or node.rssi < config["sens"]:
             
            #rho = self.rho_compute(new_sf, sf_dist[new_sf - 7])
            #if rho > 0 and u_ch[new_ch] > 0:
                #reward = - self.reward_function(rho, u_ch[new_ch], new_tp)
            #else:
                #reward = - self.reward_function_zero(new_tp)
            reward = -1

        else: 
            rho = self.rho_compute(new_sf, sf_dist[new_sf - 7])
            old_rho = self.rho_compute(old_sf, sf_dist[old_sf - 7])
            if new_sf > old_sf:
                if old_rho > rho_max and rho < rho_max :
                    self.NODES[node_index].sf = new_sf
                    self.NODES[node_index].freq = ch[new_ch]
                    self.NODES[node_index].tx = new_tp
                    
                    self.NODES[node_index].packet.sf = new_sf
                    self.NODES[node_index].packet.freq = ch[new_ch]
            
                    
                    self.NODES[node_index].update_rectime(new_sf)

                    sf_dist[old_sf - 7] = sf_dist[old_sf - 7] - 1
                    sf_dist[new_sf - 7] = sf_dist[new_sf - 7] + 1

                    old_tp_index = self.TP_SET.index(old_tp)
                    new_tp_index = self.TP_SET.index(new_tp)
                    tp_dist[old_tp_index] = tp_dist[old_tp_index] - 1
                    tp_dist[new_tp_index] = tp_dist[new_tp_index] + 1

                    old_ch_index = ch.index(old_ch)
                    new_ch_index = new_ch
                    u_ch[old_ch_index] = u_ch[old_ch_index] - 1
                    u_ch[new_ch_index] = u_ch[new_ch_index] + 1
            else:
                
                self.NODES[node_index].sf = new_sf
                self.NODES[node_index].freq = ch[new_ch]
                self.NODES[node_index].tx = new_tp
                self.NODES[node_index].update_rectime(new_sf)
                
                self.NODES[node_index].packet.sf = new_sf
                self.NODES[node_index].packet.freq = ch[new_ch]

                sf_dist[old_sf - 7] = sf_dist[old_sf - 7] - 1
                sf_dist[new_sf - 7] = sf_dist[new_sf - 7] + 1

                old_tp_index = self.TP_SET.index(old_tp)
                new_tp_index = self.TP_SET.index(new_tp)
                tp_dist[old_tp_index] = tp_dist[old_tp_index] - 1
                tp_dist[new_tp_index] = tp_dist[new_tp_index] + 1

                old_ch_index = ch.index(old_ch)
                new_ch_index = new_ch
                u_ch[old_ch_index] = u_ch[old_ch_index] - 1
                u_ch[new_ch_index] = u_ch[new_ch_index] + 1
        

            if rho > 0 and u_ch[new_ch] > 0:
                reward = self.reward_function(rho, u_ch[new_ch], new_tp)
            else:
                reward = self.reward_function_zero(new_tp)

           
        # print(sf_dist)
        # print(tp_dist)
        # print(u_ch)

        # print('Reward = ',reward)
        new_state = np.concatenate((sf_dist, tp_dist, u_ch))
        # print('')
        done = False
        self.condition = self.condition + 1
        if self.condition == 100:
            done = True
        #if der > 95:
            #done = True

        info = ""

        return new_state, reward, done, info

    def render(self):
        pass

    def close(self):
        pass


In [9]:
def save_nodes (nodes,file_name):
    pickle.dump(nodes, open("data/"+file_name, "wb"))
    print('All Nodes Were Saved !')
    
def load_nodes (file_name):
    print('Loading Nodes From File')
    nodes = pickle.load(open("data/"+file_name,"rb"))
    return nodes



In [10]:


nodes = node_Gen(5000, display=False)
nodes, sf_distribution, ch_u = SF_alloc_ch_utilization(nodes)
save_nodes(nodes,'nodes_{}.alloc'.format(len(nodes)))
#train_episodes = 300
#test_episodes = 100

train_episodes = 300
test_episodes = 100


class DQNAgent:
    def create_model(self, action_shape, state_shape):
        model = Sequential()
        learning_rate = 0.001
        init = tf.keras.initializers.HeUniform()

        model.add(
            Dense(
                256, input_shape=state_shape, activation="relu", kernel_initializer=init
            )
        )
        model.add(Dense(256, activation="relu", kernel_initializer=init))

        model.add(Dense(450000, activation="relu", kernel_initializer=init))

        model.compile(
            loss=tf.keras.losses.Huber(),
            optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
            metrics=["accuracy"],
        )
        model.save('lora_rl_model.h5')
        return model

    def get_qs(self, model, state, step):
        return model.predict(state.reshape([1, state.shape[0]]))[0]

    def train(self, env, replay_memory, model, target_model, done):
        learning_rate = 0.7  # Learning rate
        discount_factor = 0.618

        MIN_REPLAY_SIZE = 1000
        if len(replay_memory) < MIN_REPLAY_SIZE:
            return

        batch_size = 64 * 2
        mini_batch = random.sample(replay_memory, batch_size)
        # print(len(mini_batch))
        # print(mini_batch)

        # current_states = np.array([transition[0] for transition in mini_batch],dtype=object)
        # print()
        current_states = np.array(
            np.array(
                [
                    transition[0]
                    for transition in mini_batch
                    if transition[0] is not None
                ]
            )
        )
        # print('states len : ',len(current_states))
        # current_states= np.array(current_states).astype("float32")
        # print(current_states)
        # current_states = np.asarray(current_states).astype(np.float32)
        current_qs_list = model.predict(current_states)
        # print(current_qs_list)
        # print('Current: ',len(current_qs_list))
        # print('Mini batch : ',len(mini_batch))
        new_current_states = np.array(
            [transition[3] for transition in mini_batch if transition[3] is not None]
        )
        future_qs_list = target_model.predict(new_current_states)
        # print()

        X = []
        Y = []
        #print(mini_batch)
        #print(len(mini_batch))
        for index, (observation, action, reward, new_observation, done) in enumerate(
            mini_batch
        ):
            if not done:
                max_future_q = reward + discount_factor * np.max(future_qs_list[index])
            else:
                max_future_q = reward

            current_qs = current_qs_list[index]
            #print('The current_qs : ',current_qs)
            #print('Action : ',action)
            #print(type(action))
            #print(env.ACTION_SPACE_GEN)
            action_index = env.ACTION_SPACE_GEN.index(action)
            current_qs[action_index] = (1 - learning_rate) * current_qs[
                action_index
            ] + learning_rate * max_future_q

            X.append(observation)
            Y.append(current_qs)
        model.fit(
            np.array(X), np.array(Y), batch_size=batch_size, verbose=0, shuffle=True
        )


env = Environment(nodes, sf_distribution, ch_u)


agent = DQNAgent()


# agent.train()


def main():
    epsilon = 1  # Epsilon-greedy algorithm in initialized at 1 meaning every step is random at the start
    max_epsilon = 1  # You can't explore more than 100% of the time
    min_epsilon = 0.01  # At a minimum, we'll always explore 1% of the time
    decay = 0.01
    final_rewards = []
    # 1. Initialize the Target and Main models
    # Main Model (updated every 4 steps)
    print(env.observation_space.shape)
    print(env.action_space.shape)
    #print_nodes(nodes)
    #SF_alloc_plot(nodes,10,5,display=True)
    model = agent.create_model(env.action_space.shape, env.observation_space.shape)
    print("Model Created !")
    # Target Model (updated every 100 steps)
    target_model = agent.create_model(
        env.action_space.shape, env.observation_space.shape
    )
    print("Target Model Created !")
    target_model.set_weights(model.get_weights())

    replay_memory = deque(maxlen=50_000)

    target_update_counter = 0

    # X = states, y = actions
    X = []
    y = []

    steps_to_update_target_model = 0

    for episode in range(train_episodes):
        print('Episode : ',episode)
        total_training_rewards = 0
        observation = env.reset()
        done = False
        while not done:
            steps_to_update_target_model += 1
            # if True:
            # env.render()

            random_number = np.random.rand()
            # 2. Explore using the Epsilon Greedy Exploration Strategy
            if random_number <= epsilon:
                # Explore
                # print('Explore')
                action = env.action_space.sample()
                a =[]
                a.append(action[0])
                a.append(action[1])
                a.append(action[2])
                a.append(action[3])
                action = a
            else:

                # Exploit best known action
                # model dims are (batch, env.observation_space.n)
                # print('Exploit')
                encoded = observation
                encoded_reshaped = encoded.reshape([1, encoded.shape[0]])
                # print('model : ',model)
                # print('Encoded state : ',encoded_reshaped)
                predicted = model.predict(encoded_reshaped)
                action = np.argmax(predicted)
                action = env.ACTION_SPACE_GEN[action]
            #print(action)
            #print(type(action))
            new_observation, reward, done, info = env.step(action)
            replay_memory.append([observation, action, reward, new_observation, done])

            # 3. Update the Main Network using the Bellman Equation
            if steps_to_update_target_model % 4 == 0 or done:
                agent.train(env, replay_memory, model, target_model, done)

            observation = new_observation
            total_training_rewards += reward

            if done:
                print(
                    "Total training rewards: {} after n steps = {} with final reward = {}".format(
                        total_training_rewards, episode, reward
                    )
                )
                final_rewards.append(total_training_rewards)
                total_training_rewards += 1

                if steps_to_update_target_model >= 100:
                    print("Copying main network weights to the target network weights")
                    target_model.set_weights(model.get_weights())
                    steps_to_update_target_model = 0
                break

        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay * episode)
        #SF_alloc_plot(nodes,10,5,display=True)
    env.close()
    save_nodes(env.NODES,'nodes_{}.rl'.format(len(env.NODES)))
    #print_nodes(env.NODES)
    print(final_rewards)
    #print_nodes(nodes)

All Nodes Were Saved !


In [11]:
main()

(14,)
(4,)


/home/mhamnache/anaconda3/lib/python3.7/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model Created !
Target Model Created !
Episode :  0
Total training rewards: 33.994493915784226 after n steps = 0 with final reward = 0.5014274230691593
Copying main network weights to the target network weights
Episode :  1
Total training rewards: 32.84614918880318 after n steps = 1 with final reward = 0.7602185093336682
Copying main network weights to the target network weights
Episode :  2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimenta

KeyboardInterrupt: 

In [ ]:

file_names = ['nodes_5000.alloc','nodes_5000.rl']
for i in range (2):
    file_name = file_names [i]
    nodes = load_nodes(file_name)
    sf_distribution, tp_dist, u_ch = compute_sf_ch_utilization(nodes)
    def transmit(env, node):

        """transmit

        Parameters
                #print ("frequency coll 125")
        ----------
            env : simpy.core.environment
            node
        """
        while True:

            r = random.expovariate(1.0 / float(AVGSENDTIME))
            # r = random.expovariate(1.0 / float(avg_gen(node, AVGSENDTIME)))
            # node.freq = random.choice(ch)
            # node.packet.freq =node.freq
            # get_next_ch(node)
            # print('duty cycle : ',r-(node.rectime*99))
            # if (r-(node.rectime*99) < 0):
            # r = node.rectime*99
            yield env.timeout(r)
            """print(
                "Node-ID :{} SF :{} TX :{} Channel:{}  ".format(
                    node.id, node.sf, node.tx, node.freq
                )
            )"""
            node.sent = node.sent + 1
            if node in packetsAtBS:
                print("ERROR: packet already in")
            else:
                sensitivity = config_dict[node.sf - 7]["sens"]
                # print("node id: {} sf :{} s: {} ".format(node.id,node.sf,sensitivity))
                if node.packet.rssi < sensitivity:
                    node.packet.lost = True
                else:
                    node.packet.lost = False
                    if (
                        checkcollision(
                            env,
                            node.packet,
                            packetsAtBS,
                            maxBSReceives,
                            full_collision,
                        )
                        == 1
                    ):
                        node.packet.collided = 1
                    else:
                        node.packet.collided = 0
                        node.received = node.received + 1
                    packetsAtBS.append(node)
                    node.packet.addTime = env.now
            # print('rectime : ',node.packet.rectime)
            yield env.timeout(node.packet.rectime)

            if node.packet.lost:
                global nrLost
                nrLost += 1
            if node.packet.collided == 1:
                global nrCollisions
                nrCollisions = nrCollisions + 1
            if not node.packet.collided and not node.packet.lost:
                global nrReceived
                nrReceived = nrReceived + 1
            if node.packet.processed == 1:
                global nrProcessed
                nrProcessed = nrProcessed + 1
            if node in packetsAtBS:
                packetsAtBS.remove(node)
                # reset the packet
            node.packet.collided = 0
            node.packet.processed = 0
            node.packet.lost = False

    MODEL = 6
    packetsAtBS = []
    sim_env = simpy.Environment()
    bsId = 1   
    maxBSReceives = 8
    nrCollisions = 0
    nrReceived = 0
    nrProcessed = 0
    nrLost = 0
    full_collision = 1
    # AVGSENDTIME = 800
    simtime = 7200
    # print(NODES)
    for n in nodes:
        # print(n.packet)
        sim_env.process(transmit(sim_env, n))
    print("Simulation Started")
    sim_env.run(until=simtime)
    print("Simulation Finished")

    sent = sum(n.sent for n in nodes)
    energy = (
        sum(
            node.packet.rectime * TX[int(node.tx) + 2] * 3 * node.sent
            for node in nodes
        )
        / 1e6
    )

    print("Sent :", sent)
    print("Collisions :", nrCollisions)
    print("Received :", nrReceived)
    print("Processed :", nrProcessed)

    print("Loss : ", nrLost)
    # print(energy)
    der1 = (float(nrReceived) / float(sent)) * 100 if sent != 0 else 0
    print("DER1 : {:.2f} %".format(der1))
    der2 = (float(sent - nrCollisions) / float(sent)) * 100 if sent != 0 else 0
    print("DER2 : {:.2f} %".format(der2))
    print("Energy : {} J".format(energy))

    DER, DERs, charges = mathematical_model(
        len(nodes), sf_distribution, AVGSENDTIME
    )
    print(charges)
    DER = DER * 100
    print("DER ==>", DER)
    print(DERs)
    print(charges)
    error = abs(DER - der1)
    print("Error : {:.2f}".format(error))

    data = {
        "Experience": 1,
        "Rho_max":rho_max,
        "Packet_generation":p,
        "Nodes": len(nodes),
        "PathLoss": MODEL,
        "Packet_Sent": sent,
        "Packet_Loss": nrLost,
        "Collisions": nrCollisions,
        "Energy": energy,
        "DER": der1,
        "Mathematical-Model": DER,
        "Error": error,
        "SF7": charges[0],
        "SF8": charges[1],
        "SF9": charges[2],
        "SF10": charges[3],
        "SF11": charges[4],
        "SF12": charges[5],
        "DSF7": sf_distribution[0],
        "DSF8": sf_distribution[1],
        "DSF9": sf_distribution[2],
        "DSF10": sf_distribution[3],
        "DSF11": sf_distribution[4],
        "DSF12": sf_distribution[5],
       }

    finalReport(data)

In [ ]:
#devices = load_nodes('nodes_1000.alloc')
#print_nodes(devices)
#model = agent.create_model(env.action_space.shape, env.observation_space.shape)